In [1]:
import torch
print(torch.__version__)

2.5.1


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MovieRecSys").getOrCreate()
print(spark.version)

4.0.0


In [16]:
from pyspark.sql.functions import col

# Read ratings.csv
df = spark.read.csv("ratings.csv", header=True, inferSchema=True)

# Select first 1000 unique users
unique_users = df.select("userId").distinct().limit(50)

# Keep only rows from these users
df_limited = df.join(unique_users, on="userId", how="inner")

df_limited.show(5)
print("Rows after limiting:", df_limited.count())
print("Unique users:", df_limited.select("userId").distinct().count())

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|    31|     47|   5.0|
|    31|     50|   4.0|
|    31|    296|   4.5|
|    31|    318|   5.0|
|    31|    608|   4.5|
+------+-------+------+
only showing top 5 rows
Rows after limiting: 8683
Unique users: 50


In [17]:
from pyspark.ml.feature import StringIndexer, MinMaxScaler, VectorAssembler
from pyspark.sql.functions import col, round as spark_round
from pyspark.ml.functions import vector_to_array

# Encode users
user_indexer = StringIndexer(inputCol="userId", outputCol="user")
df_small = user_indexer.fit(df_limited).transform(df_limited)

# Encode movies
movie_indexer = StringIndexer(inputCol="movieId", outputCol="movie")
df_small = movie_indexer.fit(df_small).transform(df_small)

# Min-Max on Ratings
assembler = VectorAssembler(inputCols=["rating"], outputCol="rating_vec")
df_vec = assembler.transform(df_small)

scaler = MinMaxScaler(inputCol="rating_vec", outputCol="rating_scaled")
scaler_model = scaler.fit(df_vec)
scaled_df = scaler_model.transform(df_vec)

# Flatten vector -> float, then round to 1 decimal
scaled_df = scaled_df.withColumn(
    "rating_scaled",
    spark_round(vector_to_array(col("rating_scaled"))[0], 1)
)

# Keep only necessary columns
df_final = scaled_df.select("user", "movie", "rating_scaled")
df_final.show(5)


+----+-----+-------------+
|user|movie|rating_scaled|
+----+-----+-------------+
|31.0| 15.0|          1.0|
|31.0|  7.0|          0.8|
|31.0|  3.0|          0.9|
|31.0|  4.0|          1.0|
|31.0| 12.0|          0.9|
+----+-----+-------------+
only showing top 5 rows


In [18]:
# 80% train, 20% test
train_df, test_df = df_final.randomSplit([0.8, 0.2], seed=42)

#Converting PySpark DF to Pandas to Tensors
train_pd = train_df.toPandas()
test_pd = test_df.toPandas()

X_train = torch.tensor(train_pd[["user", "movie"]].values)
y_train = torch.tensor(train_pd["rating_scaled"].values)

X_test = torch.tensor(test_pd[["user", "movie"]].values)
y_test = torch.tensor(test_pd["rating_scaled"].values)

print("Train count:", X_train.shape[0])
print("Train Label:",y_train.shape[0])
print("Test count:", X_test.shape[0])
print("Test Label:",y_test.shape[0])

Train count: 7010
Train Label: 7010
Test count: 1673
Test Label: 1673


In [19]:
from torch.utils.data import TensorDataset, DataLoader

# Create datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
#print(train_dataset[2200])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [44]:
import torch
import torch.nn as nn

class CollabFiltering(nn.Module):
    def __init__(self, n_users, n_movies, emb_dim, hidden, dropout_p):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.movie_emb = nn.Embedding(n_movies, emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(emb_dim * 2, hidden),  # user+movie embeddings concatenated
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(hidden, 1),
            nn.Sigmoid()  # predict rating
        )

    def forward(self,user,movie):
        u = self.user_emb(user)
        m = self.movie_emb(movie)
        
        # Concatenate embeddings (instead of dot product)
        x = torch.cat([u, m], dim=1)
        
        # Pass through MLP
        return self.mlp(x).squeeze()

n_users = df_final.select("user").distinct().count()
n_movies = df_final.select("movie").distinct().count()

model = CollabFiltering(n_users, n_movies, emb_dim=16, hidden=32, dropout_p=0.3)

In [48]:
print(model.parameters)

<bound method Module.parameters of CollabFiltering(
  (user_emb): Embedding(50, 16)
  (movie_emb): Embedding(3524, 16)
  (mlp): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=32, out_features=1, bias=True)
    (4): Sigmoid()
  )
)>


In [49]:
loss_fn = nn.MSELoss() # regression on ratings
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=1e-5)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, weight_decay=1e-5)


In [58]:
def adam(parameter, m_state, v_state, t, lr=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
    # Loop over Tensors in model.parameters()
    for p in parameter:
        if p.grad is None:
            continue

        # First Time seeing this parameter
        if p not in m_state:
            m_state[p] = torch.zeros_like(p)
            v_state[p] = torch.zeros_like(p)

        g = p.grad
        # First moment
        m_state[p] = beta1 * m_state[p] + (1 - beta1) * g
        # Second moment
        v_state[p] = beta2 * v_state[p] + (1 - beta2) * (g * g)

        # Bias correction
        m_corrected = m_state[p] / (1 - beta1 ** t)
        v_corrected = v_state[p] / (1 - beta2 ** t)

        # Parameter update (in-place)
        p.data -= lr * m_corrected / (torch.sqrt(v_corrected) + epsilon)

    return m_state, v_state


In [59]:
n_epochs = 20
t = 0
m_state, v_state = {}, {}
train_losses, test_losses = [], []
train_rmses, test_rmses = [], []

for epoch in range(n_epochs):
    # ---------- Training ----------
    model.train()
    total_loss, total_sq_error, total_samples = 0, 0, 0
    
    for X_batch, y_batch in train_loader:
        user_batch = X_batch[:, 0].long()
        movie_batch = X_batch[:, 1].long()
        
        preds = model(user_batch, movie_batch).squeeze()
        loss = loss_fn(preds, y_batch.float())
        
        #optimizer.zero_grad()
        model.zero_grad()
        
        loss.backward()
        # Adam
        t += 1
        m_state, v_state = adam(model.parameters(), m_state, v_state,t)
        
        total_loss += loss.item()
        total_sq_error += torch.sum((preds - y_batch) ** 2).item()
        total_samples += len(y_batch)
    
    train_loss = total_loss / len(train_loader)
    train_rmse = (total_sq_error / total_samples) ** 0.5
    train_losses.append(train_loss)
    train_rmses.append(train_rmse)
    
    # ---------- Testing ----------
    model.eval()
    total_loss, total_sq_error, total_samples = 0, 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            user_batch = X_batch[:, 0].long()
            movie_batch = X_batch[:, 1].long()
            
            preds = model(user_batch, movie_batch).squeeze()
            loss = loss_fn(preds, y_batch.float())
            
            total_loss += loss.item()
            total_sq_error += torch.sum((preds - y_batch) ** 2).item()
            total_samples += len(y_batch)
    
    test_loss = total_loss / len(test_loader)
    test_rmse = (total_sq_error / total_samples) ** 0.5
    test_losses.append(test_loss)
    test_rmses.append(test_rmse)
    
    # ---------- Summary ----------
    print(f"\nEpoch {epoch+1}/{n_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train RMSE: {train_rmse:.4f}")
    print(f"Test  Loss: {test_loss:.4f}, Test  RMSE: {test_rmse:.4f}")



Epoch 1/20
Train Loss: 0.0102, Train RMSE: 0.1012
Test  Loss: 0.0695, Test  RMSE: 0.2516

Epoch 2/20
Train Loss: 0.0099, Train RMSE: 0.0993
Test  Loss: 0.0708, Test  RMSE: 0.2536

Epoch 3/20
Train Loss: 0.0099, Train RMSE: 0.0993
Test  Loss: 0.0711, Test  RMSE: 0.2554

Epoch 4/20
Train Loss: 0.0102, Train RMSE: 0.1006
Test  Loss: 0.0689, Test  RMSE: 0.2522

Epoch 5/20
Train Loss: 0.0099, Train RMSE: 0.0994
Test  Loss: 0.0697, Test  RMSE: 0.2532

Epoch 6/20
Train Loss: 0.0100, Train RMSE: 0.1001
Test  Loss: 0.0688, Test  RMSE: 0.2518

Epoch 7/20
Train Loss: 0.0098, Train RMSE: 0.0989
Test  Loss: 0.0700, Test  RMSE: 0.2533

Epoch 8/20
Train Loss: 0.0098, Train RMSE: 0.0992
Test  Loss: 0.0699, Test  RMSE: 0.2528

Epoch 9/20
Train Loss: 0.0097, Train RMSE: 0.0982
Test  Loss: 0.0711, Test  RMSE: 0.2553

Epoch 10/20
Train Loss: 0.0097, Train RMSE: 0.0985
Test  Loss: 0.0705, Test  RMSE: 0.2549

Epoch 11/20
Train Loss: 0.0092, Train RMSE: 0.0959
Test  Loss: 0.0711, Test  RMSE: 0.2553

Epoch 1